#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "cHv2ftIYyvvzPIHatIJbQZ8ImSducJGg"

##0-3.Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
!pip install -qU openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[{"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
     {"role": "user", "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。"}],
)

print(chat_completion.choices[0].message.content)

 1


#1.データ生成処理の実行

##1-1.プロンプト準備（関係抽出）

言語理解でのプロンプト例

In [ ]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、10種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
1.文章を読み、人物、場所、組織などのエンティティを名詞や固有名詞から特定してください。
2.エンティティ間の関係を識別する際には、動詞（「働いている」など）、前置詞（「で」、「に」など）、または接続詞を関係を示すキーワードとして使用してください。
3.各エンティティペアとその間の関係を括弧で囲み、カンマで区切って列挙してください。動詞が直接的なアクションを示す場合、その動詞を使用し、前置詞や接続詞が場所や方法を指定する場合、それを利用してください。
4.関係が不明確または文脈上不確かな場合は、そのエンティティペアはリストに含めないでください。
5.正確なエンティティと関係の識別には文の全体的な文脈を理解し、適切な品詞を使用して関係を表現することが求められます。
6.日本語だけを使った文章が求められます。
7.ビジネス文書やニュース記事の文章を想定しています。

多様性を与えるヒントは以下のとおりです。:
1.文章の種類：ニュース記事、ビジネス文書、技術レポート、学術論文、ブログ記事、SNS投稿など、異なるジャンルの文章を含める。
2.エンティティの種類：人物（政治家、スポーツ選手、科学者など）、場所（都市、国、建物など）、組織（企業、団体、政府機関、プロジェクトなど）などを含める。
3.関係の種類：職業、役職、勤務地、目的、手段、期間、結果、条件など、さまざまな関係を網羅する。
4.長さと複雑さ：簡単な文章から複雑な文章まで、バリエーションを持たせる。
5.文体：フォーマル、インフォーマル、説明など、異なる文体の文章を含める。
6.時制：現在、過去、未来の時制に配慮した文章を含める。
7.言語の多様性：エンティティ関係が異なる言い回しで表現した文章を含める。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON):
[
  {
    "id": 1,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 2,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 3,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 4,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 5,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 6,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 7,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 8,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 9,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  },
  {
    "id": 10,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "<入力文章>",
    "output": "<エンティティ, 関係, エンティティ>"
  }
]
"""

In [ ]:
FEWSHOT_EXAMPLES = [
  {
    "id": 1,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "大谷翔平はロサンゼルス・ドジャースのプロ野球選手であり、ロサンゼルスに住んでいます。",
    "output": "(大谷翔平, 職業, プロ野球選手)\n(大谷翔平, 勤務, ロサンゼルス・ドジャース)\n(大谷翔平, 住む, ロサンゼルス)"
  },
  {
    "id": 2,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "孫正義はソフトバンクグループ株式会社の代表取締役会長兼社長であり、佐賀県鳥栖市出身の実業家です。",
    "output": "(孫正義, 役職, 代表取締役会長兼社長)\n(孫正義, 勤務, ソフトバンクグループ株式会社)\n(孫正義, 出身, 佐賀県鳥栖市)\n(孫正義, 職業, 実業家)"
  },
  {
    "id": 3,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "松尾研は「知能を創る」ことを目指し、ウェブ研究から始まり、企業データ、ディープラーニングなどの先駆的な研究を行っています。",
    "output": "(松尾研, 目的, 知能を創る)\n(松尾研, 研究, ウェブ)\n(松尾研, 研究, 企業データ)\n(松尾研, 研究, ディープラーニング)"
  },
  {
    "id": 4,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "新型コロナウイルスは2019年に中国で発見されて全世界に拡散しました。",
    "output": "(新型コロナウイルス, 時期, 2019年)\n(新型コロナウイルス, 場所, 中国)\n(新型コロナウイルス, 拡散, 全世界)"
  },
  {
    "id": 5,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "APEC首脳会議等に出席するためサンフランシスコを訪問中の岸田文雄内閣総理大臣は、グーグル社のスンダー・ピチャイCEOと会談をしました。",
    "output": "(岸田文雄, 目的, APEC首脳会議)\n(岸田文雄, 訪問, サンフランシスコ)\n(岸田文雄, 役職, 内閣総理大臣)\n(岸田文雄, 会談, スンダー・ピチャイ)\n(スンダー・ピチャイ, 勤務, グーグル社)\n(スンダー・ピチャイ, 役職, CEO)"
  },
  {
    "id": 6,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "サイバーエージェントはAI研究開発組織を設立し、数多くの論文を発表しています。",
    "output": "(サイバーエージェント, 設立, AI研究開発組織)\n(AI研究開発組織, 発表, 論文)"
  },
  {
    "id": 7,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "日本サッカー協会は、サッカーを通じて豊かなスポーツ文化を創造し、人々の心身の健全な発達と社会の発展に貢献することを理念としています。",
    "output": "(日本サッカー協会, 理念, 豊かなスポーツ文化の創造)\n(日本サッカー協会, 手段, サッカー)\n(日本サッカー協会, 目的, 心身の健全な発達)\n(日本サッカー協会, 目的, 社会の発展の貢献)"
  },
  {
    "id": 8,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "日本政府は、アップルとグーグルが寡占するスマートフォンのモバイルエコシステムを規制する「スマホソフトウェア競争促進法案」を国会に提出しました。",
    "output": "(日本政府, 提出, スマホソフトウェア競争促進法案)\n(アップル, 状態, 寡占)\n(グーグル, 状態, 寡占)\n(スマホソフトウェア競争促進法案, 目的, モバイルエコシステムを規制)\n(スマホソフトウェア競争促進法案, 提出, 国会)"
  },
  {
    "id": 9,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "東京メトロ東西線は、線路の切り替え工事のため5月11日から12日に東陽町駅から西葛西駅間で終日運休し、振替輸送が行われます。",
    "output": "(東京メトロ東西線, 理由, 線路の切り替え工事)\n(線路の切り替え工事, 期間, 5月11日から12日)\n(東京メトロ東西線, 区間, 東陽町駅から西葛西駅間)\n(線路の切り替え工事, 手段, 振替輸送)"
  },
  {
    "id": 10,
    "instruction": "以下の文章から、エンティティ間の関係を見つけてエンティティペアと関係の種類を示してください。",
    "input": "浅草は隅田川のほとりに栄える街であり、雷門がある浅草寺を中心としたエリアです。",
    "output": "(浅草, 位置, 隅田川のほとり)\n(浅草, 特徴, 栄える街)\n(戦争時, 保有, 雷門)\n(浅草, 中心, 浅草寺)"
  }
]

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE=1 #temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS=60000
LOOP_NUM = 10 #1回あたり10件データを生成

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[{"role": "system", "content": "あなたは日本語の学習データセットを生成する専門家です。"},
         {"role": "user", "content": PROMPT_TEMPLATE}],
        temperature=TEMPERATURE,#temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,#0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,#-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3 #-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
        )

    output = chat_completion.choices[0].message.content

    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            #output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

##1-3.データの保存

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel(G_FOLDER + filename, index=False)

print(f"File saved as: {filename}")